In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
os.add_dll_directory('C:\Aorda\PSG\lib')
import psgpython as psg
import random
import yfinance as yf
%matplotlib inline
from sklearn.svm import NuSVR

allowExternal = True
suppressMessages = True

np.random.seed(42)

N = 1000
D = 4 

X = np.random.uniform(0,10, size=(N,D))
true_coeff = np.array([2.0,-3.5,1.0,4.0])
noise = np.random.normal(0,1,N)
y = X @ true_coeff + noise

In [19]:
alpha = .6
c = 1

nusvr_model = NuSVR(kernel = 'linear', C=c, nu = 1-alpha, tol=1e-7, gamma='auto')
nusvr_model.fit(X, y)

nusvr_coeff = nusvr_model.coef_[0]
intercept = nusvr_model.intercept_[0]

## PSG

In [11]:
x1 = np.ones((X.shape[0], 1))
matrix_scenarios_body = np.column_stack((x1, X, y.reshape(-1,1)))
header_primal = ["x0"]+[f"x{i+1}" for i in range(D)]
matrix_scenarios = [header_primal +["scenario_benchmark"], matrix_scenarios_body]

reg = 1/(2*c*X.shape[0])
matrix_quadratic_body = np.zeros((D+1, D+1))
matrix_quadratic_body[1:,1:] = reg
matrix_quadratic = [header_primal, matrix_quadratic_body]


In [12]:
problem_name_p = "problem_multivariate"
problem_statement_p = "minimize\n\
%f" % (1-alpha) + "*cvar_risk(" + "%f" % (alpha) + ",abs(matrix_scenarios))\n\
+quadratic(matrix_quadratic)\n\
Value: \n\
var_risk(" + "%f" % (alpha) + ",abs(matrix_scenarios))\n\
Solver: van"
problem_dictionary_primal = {'problem_name': problem_name_p, 'problem_statement': problem_statement_p,
                             'matrix_scenarios': matrix_scenarios, 'matrix_quadratic': matrix_quadratic}

# Optimization Output
res_primal = psg.psg_solver(problem_dictionary_primal, allowExternal, suppressMessages)

Problem: problem_1, solution_status = optimal
Timing: data_loading_time = 0.10, preprocessing_time = 0.01, solving_time = 0.01
Variables: optimal_point = point_problem_1
Objective: objective = 0.576155687021 [3.330669073875E-16]
Function: cvar_risk(0.6,abs(matrix_scenarios)) =  1.425333549698E+00
Function: quadratic(matrix_quadratic) =  6.022267141694E-03
Function: var_risk(0.6,abs(matrix_scenarios)) =  8.733609503508E-01
OK. Solver Finished



## Results

#### PSG/CVaR Regression

##### CVaR Coefficients

In [13]:
res_primal['point_problem_1'][1][1:]

array([ 1.99292758, -3.52867739,  0.98788926,  4.01838419])

##### CVaR Intercept

In [15]:
res_primal['point_problem_1'][1][0]

0.13962107610377297

##### NuSVR Coefficients

In [14]:
nusvr_coeff

array([ 1.99379647, -3.52810274,  0.98760498,  4.01559553])

##### NuSVR Intercept

In [20]:
intercept

0.1502509449730346